In [5]:
# --- Install necessary packages ---
!pip install numpy pandas scipy scikit-learn statsmodels patsy matplotlib seaborn google-colab dash dash-cytoscape dash-bootstrap-components

# --- Imports ---
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import random
from collections import defaultdict
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import io
#from PIL import Image  # No longer needed for direct PNG saving
import base64 # No longer needed

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Define Output Directory ---
output_dir = '/content/drive/MyDrive/data'  # Path to the 'data' folder
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# --- Explicitly set Matplotlib backend to 'Agg' for Colab ---
plt.switch_backend('Agg')

# --- Neon Theme Function (for Matplotlib/Seaborn) ---
# Modified to avoid white
def apply_minimalist_neon_theme():
    """Applies a minimalist neon theme with no filled areas and no white elements."""
    plt.style.use('dark_background')
    plt.rcParams.update({
        'axes.facecolor': '#000000',  # Black background
        'figure.facecolor': '#000000',
        'text.color': '#00FF00',  # Bright green text
        'axes.labelcolor': '#00FFFF',  # Cyan axis labels
        'xtick.color': '#00FFFF',
        'ytick.color': '#00FFFF',
        'grid.color': '#444444',  # Darker grid lines
        'lines.color': '#00FF00',
        'patch.facecolor': '#000000',  # Black patch fill
        'patch.edgecolor': '#00FFFF',  # Cyan patch outline
        'axes.edgecolor': '#00FFFF', # Cyan axes
        'boxplot.boxprops.color': '#00FFFF',
        'boxplot.whiskerprops.color': '#00FFFF',
        'boxplot.capprops.color': '#00FFFF',
        'boxplot.medianprops.color': '#FF00FF', # Magenta median
        'boxplot.flierprops.markeredgecolor': '#00FF00',
        'figure.edgecolor': '#000000', # Black figure edge
        'savefig.facecolor': '#000000', # Black background for saved figures
        'savefig.edgecolor': '#000000', # Black edge for saved figures
    })
    sns.set_style("darkgrid", {"axes.facecolor": "#000000", "grid.color": "#444444"})
    sns.set_palette(["#00FF00", "#00FFFF", "#FF00FF", "#FFFF00"])  # Neon palette


# --- Data Simulation ---
def simulate_data(n_participants=40, seed=42):
    """Simulates data, including demographics, interventions, psychological
    measures, performance, and neurophysiological data.  Effects of LLM and
    herbal blend are simulated.

    Args:
        n_participants (int): Number of participants.
        seed (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: The simulated dataset.
    """

    np.random.seed(seed)

    # Demographics
    age = np.random.randint(18, 30, size=n_participants)
    gender = np.random.choice(['Male', 'Female', 'Other'], size=n_participants)
    programming_experience = np.random.choice(
        ['Beginner', 'Intermediate', 'Advanced'], size=n_participants
    )

    # Group Assignment (balanced)
    llm_usage = np.array([1, 1, 0, 0] * (n_participants // 4))
    herbal_blend = np.array([1, 0, 1, 0] * (n_participants // 4))

    # Psychological Measures (initial and final)
    initial_self_efficacy = np.random.normal(3.5, 0.5, size=n_participants)
    initial_anxiety = np.random.normal(2.5, 0.6, size=n_participants)
    final_self_efficacy = initial_self_efficacy.copy()
    final_anxiety = initial_anxiety.copy()

    # Performance Measures
    errors_identified = np.random.randint(5, 20, size=n_participants)
    completion_time = np.random.uniform(180, 400, size=n_participants)

    # Adjust based on group (simulated effects)
    for i in range(n_participants):
        if llm_usage[i] == 1:
            final_self_efficacy[i] += 0.5
            final_anxiety[i] -= 0.4
            errors_identified[i] += 3
            completion_time[i] -= 15
        if herbal_blend[i] == 1:
            final_anxiety[i] -= 0.3
            errors_identified[i] += 1

    # Ensure reasonable bounds
    final_self_efficacy = np.clip(final_self_efficacy, 1, 5)
    final_anxiety = np.clip(final_anxiety, 1, 4)
    errors_identified = np.maximum(0, errors_identified)
    completion_time = np.maximum(60, completion_time)

    # Neurophysiological Data (simplified)
    eeg_alpha = np.random.normal(10, 2, size=n_participants)
    eeg_beta = np.random.normal(18, 3, size=n_participants)
    ecg_hr = np.random.normal(75, 10, size=n_participants)
    eda_scr = np.random.normal(0.5, 0.2, size=n_participants)
    pog_fixations = np.random.randint(20, 100, size=n_participants)
    pog_fixation_duration = np.random.uniform(200, 500, size=n_participants)
    pog_pupil_diameter = np.random.normal(3.5, 0.5, size=n_participants)
    pog_blink_rate = np.random.uniform(10, 30, size=n_participants)

    # Adjust based on group (simulated effects)
    for i in range(n_participants):
        if llm_usage[i] == 1:
            eeg_beta[i] += 2
            pog_fixations[i] -= 5
            pog_fixation_duration[i] += 50
        if herbal_blend[i] == 1:
            ecg_hr[i] -= 5
            eda_scr[i] -= 0.1

    # Create DataFrame
    data = pd.DataFrame({
        'ParticipantID': range(1, n_participants + 1),
        'Age': age,
        'Gender': gender,
        'ProgrammingExperience': programming_experience,
        'LLMUsage': llm_usage,
        'HerbalBlend': herbal_blend,
        'InitialSelfEfficacy': initial_self_efficacy,
        'FinalSelfEfficacy': final_self_efficacy,
        'InitialAnxiety': initial_anxiety,
        'FinalAnxiety': final_anxiety,
        'ErrorsIdentified': errors_identified,
        'CompletionTime': completion_time,
        'EEGAlpha': eeg_alpha,
        'EEGBeta': eeg_beta,
        'ECG_HR': ecg_hr,
        'EDA_SCR': eda_scr,
        'POGFixations': pog_fixations,
        'POGFixationDuration': pog_fixation_duration,
        'POGPupilDiameter': pog_pupil_diameter,
        'POGBlinkRate': pog_blink_rate
    })

    return data

# --- Data Preprocessing ---
def preprocess_data(data):
    """Preprocesses data: one-hot encodes categoricals, scales numericals,
    and splits into training and testing sets.

    Args:
        data (pd.DataFrame): The raw data.

    Returns:
        tuple: (X_train, X_test, y_train, y_test) preprocessed data splits.
    """

    features = data.drop(columns=['ParticipantID', 'ErrorsIdentified', 'CompletionTime'])
    performance = data[['ErrorsIdentified', 'CompletionTime']]
    features = pd.get_dummies(features, columns=['Gender', 'ProgrammingExperience'])
    numerical_features = features.select_dtypes(include=np.number).columns
    scaler = StandardScaler()
    features[numerical_features] = scaler.fit_transform(features[numerical_features])
    X_train, X_test, y_train, y_test = train_test_split(
        features, performance, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

# --- Statistical Analyses ---
def perform_statistical_analysis(data):
    """Performs descriptive stats, correlations, and group comparisons (t-tests).

    Args:
        data (pd.DataFrame): The dataset.

    Returns:
        tuple: (descriptive_stats, correlation_matrix, group_comparison_results)
    """

    descriptive_stats = data.describe()
    correlation_matrix = data[[
        'FinalSelfEfficacy', 'FinalAnxiety', 'ErrorsIdentified', 'CompletionTime'
    ]].corr()
    group_comparison_results = {}
    for variable in ['FinalSelfEfficacy', 'FinalAnxiety', 'ErrorsIdentified', 'CompletionTime']:
        llm_group = data[data['LLMUsage'] == 1][variable]
        no_llm_group = data[data['LLMUsage'] == 0][variable]
        t_stat, p_val = stats.ttest_ind(llm_group, no_llm_group)
        group_comparison_results[variable] = {'t-statistic': t_stat, 'p-value': p_val}
    return descriptive_stats, correlation_matrix, group_comparison_results


def perform_regression_analysis(X_train, y_train, dependent_variable='ErrorsIdentified'):
    """Performs regression analysis using statsmodels.

    Args:
        X_train (pd.DataFrame): Training features.
        y_train (pd.DataFrame): Training target.
        dependent_variable (str): Dependent variable to predict.

    Returns:
        statsmodels.regression.linear_model.RegressionResultsWrapper: Results.
    """

    formula = f"{dependent_variable} ~ LLMUsage + HerbalBlend + InitialSelfEfficacy + InitialAnxiety"
    y, X = dmatrices(formula, data=pd.concat([X_train, y_train], axis=1), return_type='dataframe')
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    return results

# --- Qualitative Analysis ---
def analyze_prompts(data):
    """Simulates prompt analysis, generating more realistic prompt data
    based on LLM usage and then analyzing it.  Handles potential
    ZeroDivisionError.

    Args:
        data (pd.DataFrame): The dataset.

    Returns:
        dict: Analysis results, including generated prompts.
    """
    prompts = []
    for i in range(len(data)):
        if data['LLMUsage'][i] == 1:
            # Simulate more specific prompts for LLM users
            prompt_type = random.choice(["debug", "explain", "optimize"])
            if prompt_type == "debug":
                prompts.append(f"P{i+1}: Find the error in this code: `x = 10; y = 0; z = x / y`")
            elif prompt_type == "explain":
                prompts.append(f"P{i+1}: Explain what this function does: `def add(a, b): return a + b`")
            else:  # optimize
                prompts.append(f"P{i+1}: How can I make this code faster: `for i in range(1000000): pass`")
        else:
            # Simulate more general questions for non-LLM users
            prompts.append(f"P{i+1}: I'm stuck on this task, can you give me a hint?")

    # Analyze the generated prompts
    prompt_lengths = [len(p.split()) for p in prompts]
    # Handle potential ZeroDivisionError if prompt_lengths is empty
    average_prompt_length = np.mean(prompt_lengths) if prompt_lengths else 0

    # Count keywords (more robustly)
    keyword_counts = defaultdict(int)
    for p in prompts:
        for word in p.lower().split():
            if word not in ["i", "this", "the", "a", "in", "on", "can", "you", "me", "what", "how", "is", "do", "does", "an", "here", "fix"]: # Common words
                keyword_counts[word] += 1
    most_common_keywords = sorted(keyword_counts.items(), key=lambda item: item[1], reverse=True)[:5]


    prompt_analysis = {
        "prompts": prompts,  # Include the generated prompts
        "average_prompt_length": average_prompt_length,
        "most_common_keywords": most_common_keywords,
        "question_types": ["debug", "explain", "optimize", "general help"],  # Based on simulation
    }
    return prompt_analysis


def analyze_interviews(data):
    """Simulates interview analysis, generating more realistic feedback.

    Args:
        data (pd.DataFrame): The dataset.

    Returns:
        dict: Analysis results, including generated feedback.
    """
    qualitative_feedback = []
    for i in range(len(data)):
        if data['LLMUsage'][i] == 1:
            feedback = random.choice([
                "The LLM helped me find the bug quickly.",
                "I understood the code better with the LLM's explanation.",
                "The LLM gave me suggestions I wouldn't have thought of."
            ])
        else:
            feedback = random.choice([
                "I wish I had a tool to help me understand the code.",
                "I spent a lot of time trying to find the error myself.",
                "It was difficult to debug without assistance."
            ])
        qualitative_feedback.append(f"P{i+1}: {feedback}")

    interview_analysis = {
        "perceived_usefulness_llm": np.random.uniform(3, 5) if data['LLMUsage'].any() else np.random.uniform(1, 3),
        "anxiety_reduction_llm": np.random.uniform(1, 3) if data['LLMUsage'].any() else np.random.uniform(0, 1),
        "anxiety_reduction_herbal": np.random.uniform(1, 3) if data['HerbalBlend'].any() else np.random.uniform(0, 1),
        "qualitative_feedback": qualitative_feedback,  # Include generated feedback
    }
    return interview_analysis

# --- SEM Diagram Generation (using Matplotlib) ---
def create_sem_diagram_mpl(model_name, nodes, edges, filename):
    """Creates a conceptual SEM diagram using Matplotlib, with arrowheads.

    Args:
        model_name (str): Name of the model.
        nodes (dict): Node labels and positions: {'node_label': (x, y)}.
        edges (list of tuples): Edges: [(source, target), ...].
        filename (str): Output filename.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    # Removed facecolor settings, handled by theme now
    #fig.set_facecolor('#262626')
    #ax.set_facecolor('#262626')


    # Draw nodes (perfect circles)
    for label, pos in nodes.items():
        ax.add_patch(plt.Circle(pos, 0.3, edgecolor='#00FFFF', facecolor='none', linewidth=2, zorder=2))  # Cyan outline, no fill
        ax.text(pos[0], pos[1], label, color='#00FF00', ha='center', va='center', fontsize=10, zorder=3) # Green text

    # Draw edges with arrowheads
    for source, target in edges:
        x1, y1 = nodes[source]
        x2, y2 = nodes[target]
        # Use arrow instead of line for arrowheads
        ax.arrow(x1, y1, x2 - x1, y2 - y1,
                 head_width=0.15,  # Adjust arrowhead size
                 head_length=0.2,  # Adjust arrowhead length
                 fc='#00FF00',  # Arrowhead color (green)
                 ec='#00FF00',  # Edge color (green)
                 length_includes_head=True,
                 linewidth=2, # Thicker lines
                 zorder=1)

    ax.set_title(f"SEM Model: {model_name}", color='#00FFFF')
    ax.axis('off')  # Hide axes
    plt.tight_layout()
    plt.savefig(filename)
    plt.close(fig)
    print(f"SEM diagram saved to: {filename}")


# --- Statistical Plotting Functions (Matplotlib/Seaborn) ---
def create_histogram_mpl(data, column, filename):
    """Creates a histogram with the neon theme (Matplotlib)."""
    plt.figure(figsize=(8, 6))
    # Use plt.hist for more control, no fill
    n, bins, patches = plt.hist(data[column], edgecolor='#00FFFF', facecolor='none', linewidth=2)
    plt.title(f"Histogram of {column}", color='#00FFFF') # Explicitly set title color
    plt.xlabel(column, color='#00FFFF') # Explicitly set label colors
    plt.ylabel("Frequency", color='#00FFFF')
    plt.savefig(filename)
    plt.close()
    print(f"Histogram saved to: {filename}")

def create_violin_plot_mpl(data, x_column, y_column, filename):
    """Creates a violin plot with the neon theme (Matplotlib/Seaborn)."""
    plt.figure(figsize=(8, 6))
    # Use Seaborn, but customize to remove fill
    sns.violinplot(x=data[x_column], y=data[y_column], color='#00FFFF', linewidth=2, inner=None) # No fill, cyan outline
    plt.title(f"Violin Plot of {y_column} by {x_column}", color='#00FFFF')
    plt.xlabel(x_column, color='#00FFFF')
    plt.ylabel(y_column, color='#00FFFF')
    plt.savefig(filename)
    plt.close()
    print(f"Violin plot saved to: {filename}")

def create_kde_plot_mpl(data, column1, column2, filename):
    """Creates a 2D KDE plot with the neon theme (Matplotlib/Seaborn)."""
    plt.figure(figsize=(8, 6))
    # Use Seaborn, but plot only the contours, no fill
    sns.kdeplot(x=data[column1], y=data[column2], color='#00FFFF', linewidths=2, thresh=0, levels=10) # Only contour lines
    plt.title(f"KDE Plot of {column1} vs. {column2}", color='#00FFFF')
    plt.xlabel(column1, color='#00FFFF')
    plt.ylabel(column2, color='#00FFFF')
    plt.savefig(filename)
    plt.close()
    print(f"KDE plot saved to: {filename}")

def create_stacked_bar_plot_mpl(data, x_column, y_column, color_column, filename):
    """Creates a stacked bar plot (Matplotlib/Seaborn)."""
    plt.figure(figsize=(8, 6))
    pivot_data = data.groupby([x_column, color_column])[y_column].mean().unstack()
    # Plot with no fill, only outlines
    pivot_data.plot(kind='bar', stacked=True, edgecolor='#00FFFF', linewidth=2, ax=plt.gca(), legend=False)
    plt.title(f"Stacked Bar Plot of {y_column} by {x_column} and {color_column}", color='#00FFFF')
    plt.xlabel(x_column, color='#00FFFF')
    plt.ylabel(y_column, color='#00FFFF')
    #plt.legend(title=color_column) # Removed legend to simplify
    plt.savefig(filename)
    plt.close()
    print(f"Stacked bar plot saved to: {filename}")

def create_heatmap(data, filename):
    """Creates a correlation heatmap and saves it as a PNG file."""
    data_numeric = data.select_dtypes(include=np.number)
    if data_numeric.shape[1] <= 1:
        print("Not enough numeric columns to create a heatmap.")
        return

    plt.figure(figsize=(10, 8))
    # Use Seaborn, customize colormap and lines
    sns.heatmap(data_numeric.corr(), annot=True, cmap=sns.color_palette("coolwarm", as_cmap=True), fmt=".2f",
                linewidths=.5, linecolor='#00FFFF', cbar=False, annot_kws={"color": "#00FF00"}) # No colorbar
    plt.title("Correlation Heatmap", color='#00FFFF')
    plt.savefig(filename, bbox_inches='tight', transparent=True)
    plt.close()
    print(f"Heatmap saved to: {filename}")

def generate_summary_html(data, filename="summary_statistics.html"):
    """Generates descriptive statistics and saves them as an HTML file."""
    if data.empty:
        print("No data available to generate summary.")
        return

    # Select numeric columns for analysis
    data_numeric = data.select_dtypes(include=np.number)

    # Generate descriptive statistics
    summary_stats = data_numeric.describe().transpose()

    # Save to HTML
    summary_html = summary_stats.to_html(classes='table table-dark', border=0)
    with open(filename, "w") as f:
        f.write("<html><head><title>Summary Statistics</title></head><body>")
        f.write("<h1>Descriptive Statistics</h1>")
        f.write(summary_html)
        f.write("</body></html>")

    print(f"Summary statistics saved to {filename}")

# --- Dash App ---

class DashApp:
    """Creates an interactive Dash application for visualizing the data."""
    def __init__(self, data):
        self.app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
        self.app.title = "Data Visualization"
        self.data = data
        self.elements = self._get_initial_elements()
        self.next_node_id = 1  # Initialize node ID counter
        self._setup_layout()
        self._setup_callbacks()


    def _get_initial_elements(self):
        """Defines the initial nodes and edges for the Cytoscape graph."""
        return [
            {'data': {'id': 'LLM', 'label': 'LLM Usage'}, 'classes': 'concept'},
            {'data': {'id': 'Herbal', 'label': 'Herbal Blend'}, 'classes': 'concept'},
            {'data': {'id': 'SelfEfficacy', 'label': 'Self-Efficacy'}, 'classes': 'latent'},
            {'data': {'id': 'Anxiety', 'label': 'Anxiety'}, 'classes': 'latent'},
            {'data': {'id': 'Performance', 'label': 'Performance'}, 'classes': 'outcome'},
            {'data': {'source': 'LLM', 'target': 'SelfEfficacy'}, 'classes': 'directed'},
            {'data': {'source': 'LLM', 'target': 'Anxiety'}, 'classes': 'directed'},
            {'data': {'source': 'LLM', 'target': 'Performance'}, 'classes': 'directed'},
            {'data': {'source': 'Herbal', 'target': 'Anxiety'}, 'classes': 'directed'},
            {'data': {'source': 'SelfEfficacy', 'target': 'Performance'}, 'classes': 'directed'},
            {'data': {'source': 'Anxiety', 'target': 'Performance'}, 'classes': 'directed'}
        ]

    def _setup_layout(self):
        """Sets up the layout of the Dash app."""
        self.app.layout = dbc.Container([
            html.H1("Interactive Data Visualization", className="text-center text-primary my-4", style={'color': '#00FFFF'}),
            dbc.Row([
                dbc.Col([self._create_cytoscape()], width=8),
                dbc.Col([self._create_sidebar()], width=4)
            ]),
            dcc.Store(id='hot-data', data=self.data.to_dict('records')),  # Store for data
            dcc.Store(id='current-step', data=0),  # Store for layout step
            dcc.Store(id='cytoscape-elements', data=self.elements), # Store for cytoscape elements
            # Add a dummy div for the save_cytoscape_html callback
            html.Div(id='dummy-output', style={'display': 'none'})
        ], fluid=True, style={'background-color': '#000000'})

    def _create_cytoscape(self):
        """Creates the Cytoscape component for the Dash app."""
        return cyto.Cytoscape(
            id='cytoscape-hot',
            layout={'name': 'cose'},  # Default layout
            style={'width': '100%', 'height': '600px', 'background-color': '#000000'},
            elements=self.elements,
            stylesheet=self._get_default_stylesheet()
        )

    def _create_sidebar(self):
        """Creates the sidebar for the Dash app."""
        return dbc.Card([
            dbc.CardBody([
                html.Div(id='node-data-output'),
                html.Div(id='edge-data-output'),
                dbc.ButtonGroup([
                    dbc.Button("Previous", id="prev-step-button", color="primary", className="me-2"),
                    dbc.Button("Next", id="next-step-button", color="primary")
                ]),
                html.Hr(),
                html.H4("Analysis Controls", style={'color': '#00FFFF'}),
                dbc.Form([
                    dbc.Row([
                        dbc.Label("Select Analysis Type", style={'color': '#00FFFF'}),
                        dcc.Dropdown(
                            id='analysis-type-dropdown',
                            options=[
                                {'label': 'Path Analysis', 'value': 'path'},
                                {'label': 'Factor Analysis', 'value': 'factor'},
                                {'label': 'Network Analysis', 'value': 'network'}
                            ],
                            value='path',
                            style={'backgroundColor': '#000000', 'color': '#00FF00'}
                        )
                    ]),
                    html.Br(),
                    dbc.Row([
                        dbc.Label("Significance Level", style={'color': '#00FFFF'}),
                        dcc.Slider(
                            id='significance-slider',
                            min=0.01,
                            max=0.1,
                            step=0.01,
                            value=0.05,
                            marks={i/100: f'{i}%' for i in range(1, 11)},
                            className='my-2'
                        )
                    ])
                ]),
                html.Hr(),
                html.H4("Node/Edge Controls", style={'color': '#00FFFF'}),
                dbc.Form([
                    dbc.Row([
                        dbc.Col(dbc.Button("Add Node", id="add-node-button", color="success", className="me-1"), width="auto"),
                        dbc.Col(dbc.Button("Add Edge", id="add-edge-button", color="success", className="me-1"), width="auto"),
                    ]),
                    dbc.Row([
                        dbc.Col(dbc.Button("Remove Node", id="remove-node-button", color="danger", className="me-1"), width="auto"),
                        dbc.Col(dbc.Button("Remove Edge", id="remove-edge-button", color="danger", className="me-1"), width="auto"),
                    ]),
                    # Input fields for adding nodes and edges
                    dbc.Row([
                        dbc.Col(dbc.Input(id='new-node-label', placeholder='New Node Label', type='text'), width=6),
                        dbc.Col(dbc.Select(id='new-node-class', options=[
                            {'label': 'Concept', 'value': 'concept'},
                            {'label': 'Latent', 'value': 'latent'},
                            {'label': 'Outcome', 'value': 'outcome'}
                        ], placeholder='Node Class'), width=6)
                    ]),
                    dbc.Row([
                        dbc.Col(dbc.Select(id='source-node', options=[], placeholder='Source Node'), width=6),
                        dbc.Col(dbc.Select(id='target-node', options=[], placeholder='Target Node'), width=6)
                    ]),
                ]),
                html.Hr(),
                html.H4("Node Statistics", style={'color': '#00FFFF'}),
                html.Div(id='node-stats-output'),
            ])
        ], style={'backgroundColor': '#111111'})

    def _get_default_stylesheet(self):
        """Defines the default stylesheet for the Cytoscape graph."""
        return [
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)',
                    'background-color': '#000000',
                    'border-color': '#00FFFF',
                    'border-width': '2px',
                    'color': '#00FFFF',
                    'font-size': '12px',
                    'text-valign': 'center',
                    'text-halign': 'center',
                    'width': '60px',  # Consistent node size
                    'height': '60px', # Consistent node size
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'line-color': '#00FF00',
                    'target-arrow-color': '#00FF00',
                    'target-arrow-shape': 'triangle',
                    'curve-style': 'bezier',
                    'width': '2px'
                }
            },
            {
                'selector': ':selected',
                'style': {
                    'border-color': '#FF00FF',
                    'line-color': '#FF00FF',
                    'target-arrow-color': '#FF00FF'
                }
            },
            {
                'selector': '.concept',
                'style': {
                    'background-color': '#000000',
                    'border-color': '#00FFFF',
                }
            },
            {
                'selector': '.latent',
                'style': {
                    'background-color': '#000000',
                    'border-color': '#FFFF00',
                }
            },
            {
                'selector': '.outcome',
                'style': {
                    'background-color': '#000000',
                    'border-color': '#FF00FF',
                }
            },
            {
                'selector': '.directed',
                'style': {
                    'line-color': '#00FF00',
                    'target-arrow-color': '#00FF00',
                    'target-arrow-shape': 'triangle',
                }
            }
        ]

    def _setup_callbacks(self):
        """Sets up the callbacks for the Dash app."""

        # Update node and edge data display
        @self.app.callback(
            [Output('node-data-output', 'children'),
             Output('edge-data-output', 'children')],
            [Input('cytoscape-hot', 'selectedNodeData'),
             Input('cytoscape-hot', 'selectedEdgeData'),
             Input('analysis-type-dropdown', 'value'),
             Input('significance-slider', 'value')]
        )
        def update_data_display(node_data, edge_data, analysis_type, significance):
            """Updates the node and edge data display in the sidebar."""
            node_output = "No node selected"
            edge_output = "No edge selected"

            if node_data:
                node = node_data[0]  # Use the first selected node
                node_output = html.Div([
                    html.H5(f"Node: {node['label']}", style={'color': '#00FFFF'}),
                    html.P(f"Analysis Type: {analysis_type}", style={'color': '#00FF00'}),
                    html.P(f"Significance: {significance}", style={'color': '#00FF00'})
                ])

            if edge_data:
                edge = edge_data[0]  # Use the first selected edge
                edge_output = html.Div([
                    html.H5("Edge Data", style={'color': '#00FFFF'}),
                    html.P(f"Source: {edge['source']}", style={'color': '#00FF00'}),
                    html.P(f"Target: {edge['target']}", style={'color': '#00FF00'})
                ])

            return node_output, edge_output

        # Add Node
        @self.app.callback(
            Output('cytoscape-elements', 'data'),
            Input('add-node-button', 'n_clicks'),
            [State('new-node-label', 'value'),
             State('new-node-class', 'value'),
             State('cytoscape-elements', 'data')]
        )
        def add_node(n_clicks, label, node_class, elements):
            if n_clicks is None or not label or not node_class:
                return elements

            new_node_id = str(self.next_node_id)  # Use string IDs
            self.next_node_id += 1
            new_node = {'data': {'id': new_node_id, 'label': label}, 'classes': node_class}
            return elements + [new_node]

        # Add Edge
        @self.app.callback(
            Output('cytoscape-elements', 'data', allow_duplicate=True),
            Input('add-edge-button', 'n_clicks'),
            [State('source-node', 'value'),
             State('target-node', 'value'),
             State('cytoscape-elements', 'data')],
            prevent_initial_call=True
        )
        def add_edge(n_clicks, source, target, elements):
            if n_clicks is None or not source or not target:
                return elements

            new_edge = {'data': {'source': source, 'target': target}, 'classes': 'directed'}
            return elements + [new_edge]


        # Remove Node
        @self.app.callback(
            Output('cytoscape-elements', 'data', allow_duplicate=True),
            Input('remove-node-button', 'n_clicks'),
            State('cytoscape-hot', 'selectedNodeData'),
            State('cytoscape-elements', 'data'),
            prevent_initial_call=True
        )
        def remove_node(n_clicks, selected_node_data, elements):
            if n_clicks is None or not selected_node_data:
                return elements

            node_id_to_remove = selected_node_data[0]['id']
            updated_elements = [
                el for el in elements
                if el['data'].get('id') != node_id_to_remove and
                (el['data'].get('source') != node_id_to_remove and el['data'].get('target') != node_id_to_remove)
            ]
            return updated_elements

        # Remove Edge
        @self.app.callback(
            Output('cytoscape-elements', 'data', allow_duplicate=True),
            Input('remove-edge-button', 'n_clicks'),
            State('cytoscape-hot', 'selectedEdgeData'),
            State('cytoscape-elements', 'data'),
            prevent_initial_call=True
        )
        def remove_edge(n_clicks, selected_edge_data, elements):
            if n_clicks is None or not selected_edge_data:
                return elements

            edge_to_remove = selected_edge_data[0]
            updated_elements = [el for el in elements if not (
                el.get('data', {}).get('source') == edge_to_remove['source'] and
                el.get('data', {}).get('target') == edge_to_remove['target']
            )]
            return updated_elements

        # Update Cytoscape elements
        @self.app.callback(
            Output('cytoscape-hot', 'elements'),
            Input('cytoscape-elements', 'data')
        )
        def update_cytoscape(elements):
            return elements

        # Update dropdown options for source and target nodes
        @self.app.callback(
            [Output('source-node', 'options'),
             Output('target-node', 'options')],
            Input('cytoscape-elements', 'data')
        )
        def update_node_options(elements):
            node_options = [{'label': el['data']['label'], 'value': el['data']['id']} for el in elements if 'id' in el['data']]
            return node_options, node_options

        # Node Statistics
        @self.app.callback(
            Output('node-stats-output', 'children'),
            Input('cytoscape-hot', 'selectedNodeData'),
            State('hot-data', 'data')
        )
        def update_node_stats(node_data, data_dict):
            if not node_data or not data_dict:
                return "Select a node to view statistics."

            node_id = node_data[0]['id']
            node_label = node_data[0]['label']

            # Convert data_dict back to DataFrame
            df = pd.DataFrame(data_dict)

            # Map node label to DataFrame column name (if applicable)
            column_mapping = {
                'LLM Usage': 'LLMUsage',
                'Herbal Blend': 'HerbalBlend',
                'Self-Efficacy': 'FinalSelfEfficacy',  # Assuming you want final values
                'Anxiety': 'FinalAnxiety',
                'Performance': 'ErrorsIdentified'  # Example: Using ErrorsIdentified for Performance
            }
            column_name = column_mapping.get(node_label)

            if column_name and column_name in df.columns:
                # Calculate statistics
                mean_val = df[column_name].mean()
                median_val = df[column_name].median()
                std_dev = df[column_name].std()

                return html.Div([
                    html.H5(f"Statistics for {node_label}", style={'color': '#00FFFF'}),
                    html.P(f"Mean: {mean_val:.2f}", style={'color': '#00FF00'}),
                    html.P(f"Median: {median_val:.2f}", style={'color': '#00FF00'}),
                    html.P(f"Standard Deviation: {std_dev:.2f}", style={'color': '#00FF00'})
                ])
            else:
                return html.P(f"No corresponding data found for {node_label}", style={'color': '#FF0000'})


        # --- Callback to save Cytoscape as HTML ---
        @self.app.callback(
            Output('dummy-output', 'children'),  # Use a dummy output!
            Input('cytoscape-hot', 'elements'),
            prevent_initial_call=True
        )
        def save_cytoscape_html(elements):
            """Saves the current Cytoscape graph as an HTML file."""
            if not elements:
                return dash.no_update

            cyto_temp = cyto.Cytoscape(
                id='cytoscape-temp',
                elements=elements,
                stylesheet=self._get_default_stylesheet(),
                layout={'name': 'cose'}
            )
            html_str = cyto_temp.to_html()
            filename = os.path.join(output_dir, 'SEM.html')  # Save in the specified output directory
            try:
                with open(filename, "w") as f:
                    f.write(html_str)
                print(f"Cytoscape HTML saved to: {filename}")
            except Exception as e:
                print(f"Error saving Cytoscape HTML: {e}")
            return dash.no_update



    def run_server(self, debug=True, port=8051):
        """Runs the Dash app server."""
        self.app.run_server(debug=debug, port=port)

# --- (Previous code remains the same up to the DashApp class definition) ---

# --- Main Execution Block ---

if __name__ == '__main__':
    apply_minimalist_neon_theme()  # Apply the theme
    data = simulate_data()  # Generate the data
    X_train, X_test, y_train, y_test = preprocess_data(data) #preprocess

    # --- Basic Statistical Analysis and Plotting ---
    descriptive_stats, correlation_matrix, group_comparison_results = perform_statistical_analysis(data)

    # Save descriptive stats to HTML
    generate_summary_html(data, filename=os.path.join(output_dir, "summary_statistics.html"))

    # --- Create and save plots (REVISED PLOT LIST) ---

    # Violin Plots
    create_violin_plot_mpl(data, 'LLMUsage', 'CompletionTime', os.path.join(output_dir, 'llm_completion_violin.png'))
    create_violin_plot_mpl(data, 'LLMUsage', 'ErrorsIdentified', os.path.join(output_dir, 'llm_errors_violin.png'))
    create_violin_plot_mpl(data, 'HerbalBlend', 'CompletionTime', os.path.join(output_dir, 'herbal_completion_violin.png'))
    create_violin_plot_mpl(data, 'HerbalBlend', 'ErrorsIdentified', os.path.join(output_dir, 'herbal_errors_violin.png'))
    create_violin_plot_mpl(data, 'LLMUsage', 'FinalSelfEfficacy', os.path.join(output_dir, 'llm_selfefficacy_violin.png'))
    create_violin_plot_mpl(data, 'HerbalBlend', 'FinalAnxiety', os.path.join(output_dir, 'herbal_anxiety_violin.png'))

    # Stacked Bar Plots
    create_stacked_bar_plot_mpl(data, 'ProgrammingExperience', 'CompletionTime', 'LLMUsage', os.path.join(output_dir, 'experience_completion_llm_stackedbar.png'))
    create_stacked_bar_plot_mpl(data, 'ProgrammingExperience', 'ErrorsIdentified', 'LLMUsage', os.path.join(output_dir, 'experience_errors_llm_stackedbar.png'))
    create_stacked_bar_plot_mpl(data, 'ProgrammingExperience', 'FinalSelfEfficacy', 'LLMUsage', os.path.join(output_dir, 'experience_selfefficacy_llm_stackedbar.png'))
    create_stacked_bar_plot_mpl(data, 'ProgrammingExperience', 'FinalAnxiety', 'LLMUsage', os.path.join(output_dir, 'experience_anxiety_llm_stackedbar.png'))
    create_stacked_bar_plot_mpl(data, 'Gender', 'CompletionTime', 'HerbalBlend', os.path.join(output_dir, 'gender_completion_herbal_stackedbar.png'))
    create_stacked_bar_plot_mpl(data, 'Gender', 'ErrorsIdentified', 'HerbalBlend', os.path.join(output_dir, 'gender_errors_herbal_stackedbar.png'))


    # --- SEM Diagrams (Conceptual) ---

    # 1. Basic Model
    nodes_basic = {
        'LLM Usage': (1, 3),
        'Herbal Blend': (1, 1),
        'Self-Efficacy': (3, 4),
        'Anxiety': (3, 2),
        'Performance': (5, 3)
    }
    edges_basic = [
        ('LLM Usage', 'Self-Efficacy'),
        ('LLM Usage', 'Anxiety'),
        ('LLM Usage', 'Performance'),
        ('Herbal Blend', 'Anxiety'),
        ('Self-Efficacy', 'Performance'),
        ('Anxiety', 'Performance')
    ]
    create_sem_diagram_mpl("Basic Model", nodes_basic, edges_basic, os.path.join(output_dir, 'sem_diagram_basic.png'))

    # --- Regression Analysis ---
    regression_results_errors = perform_regression_analysis(X_train, y_train, 'ErrorsIdentified')
    print("\nRegression Results (Errors Identified):\n", regression_results_errors.summary())
    regression_results_time = perform_regression_analysis(X_train, y_train, 'CompletionTime')
    print("\nRegression Results (Completion Time):\n", regression_results_time.summary())

    # --- Qualitative Analyses ---
    prompt_analysis_results = analyze_prompts(data)
    print("\nPrompt Analysis:\n", prompt_analysis_results)
    interview_analysis_results = analyze_interviews(data)
    print("\nInterview Analysis:\n", interview_analysis_results)

    # --- Dash App Execution ---
    app = DashApp(data)
    app.run_server(debug=True, port=8051)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Summary statistics saved to /content/drive/MyDrive/data/summary_statistics.html
Violin plot saved to: /content/drive/MyDrive/data/llm_completion_violin.png
Violin plot saved to: /content/drive/MyDrive/data/llm_errors_violin.png
Violin plot saved to: /content/drive/MyDrive/data/herbal_completion_violin.png
Violin plot saved to: /content/drive/MyDrive/data/herbal_errors_violin.png
Violin plot saved to: /content/drive/MyDrive/data/llm_selfefficacy_violin.png
Violin plot saved to: /content/drive/MyDrive/data/herbal_anxiety_violin.png
Stacked bar plot saved to: /content/drive/MyDrive/data/experience_completion_llm_stackedbar.png
Stacked bar plot saved to: /content/drive/MyDrive/data/experience_errors_llm_stackedbar.png
Stacked bar plot saved to: /content/drive/MyDrive/data/experience_selfefficacy_llm_stackedbar.png
Stacked bar plot saved to: /content/drive/MyDrive

<IPython.core.display.Javascript object>